In [3]:
import dspy
import yaml
import pandas as pd
import os

# Load the configuration file
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)
    EMOTIONS, RANDOM_SEED = (
        config["EMOTIONS"],
        config["RANDOM_SEED"],
    )

llm = dspy.OpenAI(model='accounts/fireworks/models/llama-v3p1-70b-instruct', api_base="https://api.fireworks.ai/inference/v1/", api_key=os.environ["fireworks"])
# llm = dspy.OllamaLocal("llama3.1:8b")
dspy.settings.configure(lm=llm)

In [17]:
# Remove neutral from emotions 
EMOTIONS = [e for e in EMOTIONS if e != "neutral"]

# Prepare Dataset

In [5]:
df = pd.read_csv("../cleaning/CSVs/gt.csv")
# drop all columns except text and ground_truth
df = df[["text", "ground_truth"]]
train_df = df.iloc[:int(len(df)*0.8)]
dev_df = df.iloc[int(len(df)*0.8):]

trainset = [dspy.Example(text = text, emotions = emotions).with_inputs("text") for text, emotions in zip(train_df["text"], train_df["ground_truth"])]
devset = [dspy.Example(text = text, emotions = emotions).with_inputs("text") for text, emotions in zip(dev_df["text"], dev_df["ground_truth"])]

train_example = trainset[0]
dev_example = devset[0]

In [18]:
class Emotion(dspy.Signature):
    """Make a multilabel prediction of the emotion(s) in the text"""
    
    text = dspy.InputField(desc="The text to be classified, context should be implicitly assumed if not explicitly stated")
    emotions = dspy.OutputField(desc = f"Can be any number of emotions, or otherwise only neutral. Has to be from those emotions: {EMOTIONS}") 

In [24]:
# Define the predictor.
generate_answer = dspy.Predict(Emotion)
train_example = trainset[5].text
# Call the predictor on a particular input.
pred = generate_answer(text=train_example)
# Print the input and the prediction.
print(f"Question: {train_example}")
print(f"Predicted Answer: {pred.emotions}")

Question: A few years ago a lot of shows tried to do a �??second screen experience�?�. It didn�??t work and died a quiet death.
Predicted Answer: disappointment, disapproval, annoyance


In [32]:
#TODO Assert max len
class LEGO(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate_answer = dspy.ChainOfThought(Emotion)
    
    def forward(self, text):
        prediction = self.generate_answer(text = text)
        return dspy.Prediction(emotions = prediction.emotions)

In [33]:
# Evaluation
#TODO create a good function (This one is trivial)
def validate_emotions(example, pred, trace=None):
    return example.emotions.lower() == pred.emotions.lower()

In [27]:
from dspy.teleprompt import BootstrapFewShot
# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_emotions)

# Compile!
compiledLEGO = teleprompter.compile(LEGO(), trainset=trainset)

100%|██████████| 160/160 [05:40<00:00,  2.13s/it]

Bootstrapped 0 full traces after 160 examples in round 0.


In [37]:
test_text = "He was waiting for this moment for a long time"
normal_prediction = dspy.Predict(Emotion)
print(f"Before compiling: {normal_prediction(text=test_text).emotions}")
#print(f"After compiling: {compiledLEGO(text=test_text).emotions}")

Before compiling: excitement, anticipation, nervousness


In [36]:
llm.inspect_history(n=1)





Make a multilabel prediction of the emotion(s) in the text

---

Follow the following format.

Text: The text to be classified, context should be implicitly assumed if not explicitly stated
Reasoning: Let's think step by step in order to ${produce the emotions}. We ...
Emotions: Can be any number of emotions, or otherwise only neutral. Has to be from those emotions: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']

---

Text: He was waiting for this moment for a long time
Reasoning: Let's think step by step in order to produce the emotions. We can start by looking at the phrase "waiting for this moment". This phrase implies anticipation and expectation. The fact that he was waiting for a long time suggests that t